In [1]:
import tushare as ts
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels import regression
import ffn

E:\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
#选取可行的股票池 以 600848 为例 取 2015-01-01 到 2016-12-31的数据为训练数据  valid 2017-01-01 到 2017-3-31 test 2017-04-1-2017-8-31
train_data = ts.get_hist_data('600848',start='2015-01-05',end='2016-12-31')
valid_data = ts.get_hist_data('600848',start='2017-01-01',end='2017-03-31')
test_data = ts.get_hist_data('600848',start='2017-04-01',end='2017-08-31')

In [3]:
#筛选特征
train_data=train_data.loc[:,['open','high','close','low','price_change','p_change','turnover']]
valid_data=valid_data.loc[:,['open','high','close','low','price_change','p_change','turnover']]
test_data=test_data.loc[:,['open','high','close','low','price_change','p_change','turnover']]

In [4]:
test_data.head()

,open,high,close,low,price_change,p_change,turnover
date,,,,,,,
2017-08-31,24.16,25.88,25.62,23.78,1.47,6.09,6.49
2017-08-30,23.25,24.60,24.15,23.21,1.09,4.73,4.92
2017-08-29,23.45,23.85,23.06,23.04,-0.44,-1.87,2.28
2017-08-28,23.45,23.70,23.50,23.05,0.13,0.56,2.98
2017-08-25,22.50,23.60,23.37,22.50,0.75,3.32,3.24


In [5]:
first_test_data = test_data.loc[:,['close']]

In [6]:
second_test_data = first_test_data.shift(1)

In [7]:
first_test_data.head()

,close
date,
2017-08-31,25.62
2017-08-30,24.15
2017-08-29,23.06
2017-08-28,23.50
2017-08-25,23.37


In [8]:
second_test_data.head() #滞后一期的close

,close
date,
2017-08-31,NaN
2017-08-30,25.62
2017-08-29,24.15
2017-08-28,23.06
2017-08-25,23.50


In [9]:
simpleret = (first_test_data - second_test_data)/second_test_data #计算简单一期收益

In [10]:
test_data['simpleret'] = simpleret

In [11]:
test_data.head()

,open,high,close,low,price_change,p_change,turnover,simpleret
date,,,,,,,,
2017-08-31,24.16,25.88,25.62,23.78,1.47,6.09,6.49,NaN
2017-08-30,23.25,24.60,24.15,23.21,1.09,4.73,4.92,-0.057377
2017-08-29,23.45,23.85,23.06,23.04,-0.44,-1.87,2.28,-0.045135
2017-08-28,23.45,23.70,23.50,23.05,0.13,0.56,2.98,0.019081
2017-08-25,22.50,23.60,23.37,22.50,0.75,3.32,3.24,-0.005532


In [28]:
test_data = test_data.fillna(0)
test_data.head()

,open,high,close,low,price_change,p_change,turnover,simpleret
date,,,,,,,,
2017-08-31,24.16,25.88,25.62,23.78,1.47,6.09,6.49,0.000000
2017-08-30,23.25,24.60,24.15,23.21,1.09,4.73,4.92,-0.057377
2017-08-29,23.45,23.85,23.06,23.04,-0.44,-1.87,2.28,-0.045135
2017-08-28,23.45,23.70,23.50,23.05,0.13,0.56,2.98,0.019081
2017-08-25,22.50,23.60,23.37,22.50,0.75,3.32,3.24,-0.005532


In [30]:
ret = test_data.simpleret
model = sm.OLS(ret,sm.add_constant(test_data.iloc[:,-7:-2])).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              simpleret   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     1.440
Date:                Mon, 04 Sep 2017   Prob (F-statistic):              0.217
Time:                        00:27:18   Log-Likelihood:                 206.88
No. Observations:                 104   AIC:                            -401.8
Df Residuals:                      98   BIC:                            -385.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.0089      0.038     -0.236   